In [3]:
!pip install numpy==1.24.3

# Then install the required packages
!pip install chromadb==0.4.18 PyMuPDF==1.21.1 google-generativeai==0.3.2 python-dotenv==1.0.1 tqdm==4.66.2 sentence-transformers==2.5.1

print("✅ All dependencies installed with NumPy compatibility fix!")

  Using cached chromadb-0.4.18-py3-none-any.whl.metadata (7.4 kB)
  Using cached PyMuPDF-1.21.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
  Using cached google_generativeai-0.3.2-py3-none-any.whl.metadata (5.9 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached sentence_transformers-2.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached chroma_hnswlib-0.7.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached posthog-4.8.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached pulsar_client-3.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
  Using cached onnxruntime-1.22.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached opentelemetry_api-1.34.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.34.1-py3-none-any.whl.metadata

✅ All dependencies installed with NumPy compatibility fix!


import os
from google.colab import userdata

# Create Dataset directory
os.makedirs('/content/Dataset', exist_ok=True)
print("✅ Dataset folder created at /content/Dataset")

# Set up Google API Key from Colab secrets
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ Google API Key loaded from secrets")
except:
    print("❌ Please add your GOOGLE_API_KEY to Colab secrets")
    print("Go to the key icon (🔑) in the left sidebar and add GOOGLE_API_KEY")

In [3]:
import os
from google.colab import userdata

# Create Dataset directory
os.makedirs('/content/Dataset', exist_ok=True)
print("✅ Dataset folder created at /content/Dataset")

# Set up Google API Key from Colab secrets
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ Google API Key loaded from secrets")
except:
    print("❌ Please add your GOOGLE_API_KEY to Colab secrets")
    print("Go to the key icon (🔑) in the left sidebar and add GOOGLE_API_KEY")

✅ Dataset folder created at /content/Dataset
✅ Google API Key loaded from secrets


In [2]:
from google.colab import files
import shutil

def upload_pdfs():
    """Function to upload PDF files to the Dataset folder"""
    print("Please select your PDF files to upload...")
    uploaded = files.upload()

    pdf_count = 0
    for filename in uploaded.keys():
        if filename.endswith('.pdf'):
            # Move to Dataset folder
            shutil.move(filename, f'/content/Dataset/{filename}')
            pdf_count += 1
            print(f"✅ Moved {filename} to Dataset folder")
        else:
            print(f"⚠️ {filename} is not a PDF file, skipping...")

    print(f"\n✅ Successfully uploaded {pdf_count} PDF files to Dataset folder")

    # List files in Dataset folder
    dataset_files = os.listdir('/content/Dataset')
    pdf_files = [f for f in dataset_files if f.endswith('.pdf')]
    print(f"📁 Dataset folder now contains: {pdf_files}")

    return pdf_files

# Call the upload function
uploaded_files = upload_pdfs()

Please select your PDF files to upload...


Saving 1706.03762v7.pdf to 1706.03762v7.pdf
Saving 2005.11401v4.pdf to 2005.11401v4.pdf
Saving 2005.14165v4.pdf to 2005.14165v4.pdf
✅ Moved 1706.03762v7.pdf to Dataset folder
✅ Moved 2005.11401v4.pdf to Dataset folder
✅ Moved 2005.14165v4.pdf to Dataset folder

✅ Successfully uploaded 3 PDF files to Dataset folder
📁 Dataset folder now contains: ['2005.14165v4.pdf', '2005.11401v4.pdf', '1706.03762v7.pdf']


In [1]:
import os
import fitz
import chromadb
from chromadb.utils import embedding_functions
import google.generativeai as genai
from typing import List, Dict, Tuple
from tqdm import tqdm
from google.colab import userdata

class RAGSystem:
    def __init__(self, model_name: str = "gemini-1.5-flash"):
        print("Initializing RAG System...")
        self.initialized = False

        # Get API key from Colab secrets
        try:
            api_key = userdata.get('GOOGLE_API_KEY')
            print("✅ Google API Key retrieved from secrets")
        except:
            api_key = os.getenv("GOOGLE_API_KEY")
            if not api_key:
                print("❌ GOOGLE_API_KEY not found. Please set it in Colab secrets.")
                return

        try:
            # Initialize Gemini
            genai.configure(api_key=api_key)
            self.model = genai.GenerativeModel(model_name)
            print("✅ Gemini model initialized")

            # Initialize ChromaDB with persistent storage
            self.client = chromadb.PersistentClient(path="/content/chromadb")
            print("✅ ChromaDB client initialized")

            # Try different embedding functions in order of preference
            embedding_function = self._get_embedding_function()
            if not embedding_function:
                print("❌ Could not initialize any embedding function")
                return

            # Try to get existing collection or create new one
            try:
                self.collection = self.client.get_collection(
                    name="research_papers",
                    embedding_function=embedding_function
                )
                print("✅ Loaded existing ChromaDB collection")
            except:
                self.collection = self.client.create_collection(
                    name="research_papers",
                    embedding_function=embedding_function
                )
                print("✅ Created new ChromaDB collection")

            # Store document metadata
            self.documents_metadata = {}
            self.initialized = True
            print("🚀 RAG System ready!")

        except Exception as e:
            print(f"❌ Error initializing RAG System: {e}")
            self.initialized = False

    def _get_embedding_function(self):
        """Try different embedding functions in order of preference."""

        # Option 1: Try sentence-transformers with error handling
        try:
            print("🔄 Trying sentence-transformers embedding...")
            embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
                model_name="all-MiniLM-L6-v2"
            )
            # Test the embedding function
            test_embedding = embedding_func(["test"])
            print("✅ Sentence-transformers embedding function working")
            return embedding_func
        except Exception as e:
            print(f"⚠️ Sentence-transformers failed: {e}")

        # Option 2: Try Google's embedding (if available)
        try:
            print("🔄 Trying Google embedding...")
            embedding_func = embedding_functions.GoogleGenerativeAiEmbeddingFunction(
                api_key=userdata.get('GOOGLE_API_KEY')
            )
            print("✅ Google embedding function working")
            return embedding_func
        except Exception as e:
            print(f"⚠️ Google embedding failed: {e}")

        # Option 3: Default embedding function
        try:
            print("🔄 Trying default embedding...")
            embedding_func = embedding_functions.DefaultEmbeddingFunction()
            print("✅ Default embedding function working")
            return embedding_func
        except Exception as e:
            print(f"⚠️ Default embedding failed: {e}")

        return None

    def _check_initialized(self):
        """Check if the system is properly initialized."""
        if not self.initialized:
            print("❌ RAG System not properly initialized. Please check your setup.")
            return False
        return True

    def process_document(self, file_path: str) -> List[Dict]:
        """Process a PDF document and return chunks with metadata."""
        if not self._check_initialized():
            return []

        try:
            doc = fitz.open(file_path)
            chunks = []

            for page_num in tqdm(range(len(doc)), desc=f"Processing {os.path.basename(file_path)}"):
                page = doc[page_num]
                text = page.get_text()

                if not text.strip():  # Skip empty pages
                    continue

                # Get section titles (headers) from the page
                headers = self._extract_headers(page)

                # Split text into chunks (approximately 1000 characters with overlap)
                chunk_size = 1000
                overlap = 200

                for i in range(0, len(text), chunk_size - overlap):
                    chunk = text[i:i + chunk_size]
                    if not chunk.strip():
                        continue

                    # Find the most relevant header for this chunk
                    relevant_header = self._find_relevant_header(chunk, headers)

                    chunks.append({
                        "text": chunk.strip(),
                        "metadata": {
                            "paper_title": os.path.basename(file_path),
                            "page_number": page_num + 1,
                            "section_title": relevant_header
                        }
                    })

            doc.close()  # Close document to free memory
            return chunks

        except Exception as e:
            print(f"❌ Error processing {file_path}: {str(e)}")
            return []

    def _extract_headers(self, page) -> List[str]:
        """Extract headers from a page using font size and style."""
        headers = []
        try:
            blocks = page.get_text("dict")["blocks"]

            for block in blocks:
                if "lines" in block:
                    for line in block["lines"]:
                        for span in line["spans"]:
                            # Check if text is likely a header (larger font, bold, etc.)
                            if span["size"] > 12 and span["flags"] & 16:  # 16 is bold flag
                                text = span["text"].strip()
                                if text and len(text) > 3:  # Avoid very short headers
                                    headers.append(text)
        except:
            pass  # If header extraction fails, continue without headers

        return headers

    def _find_relevant_header(self, chunk: str, headers: List[str]) -> str:
        """Find the most relevant header for a chunk of text."""
        if not headers:
            return "Content"

        # Simple heuristic: use the last header found before this chunk
        return headers[-1] if headers else "Content"

    def add_documents(self, directory: str):
        """Add all PDF documents from a directory to the vector store."""
        if not self._check_initialized():
            return

        if not os.path.exists(directory):
            print(f"❌ Directory not found: {directory}")
            return

        pdf_files = [f for f in os.listdir(directory) if f.endswith(".pdf")]

        if not pdf_files:
            print(f"❌ No PDF files found in {directory}")
            return

        print(f"📚 Found {len(pdf_files)} PDF files to process")

        total_chunks = 0
        for filename in pdf_files:
            file_path = os.path.join(directory, filename)
            print(f"\n📖 Processing: {filename}")

            try:
                chunks = self.process_document(file_path)

                if chunks:
                    # Create unique IDs for chunks
                    existing_ids = [f"{filename}_{i}" for i in range(len(chunks))]

                    # Add chunks to ChromaDB in batches to avoid memory issues
                    batch_size = 100
                    for i in range(0, len(chunks), batch_size):
                        batch_chunks = chunks[i:i+batch_size]
                        batch_ids = existing_ids[i:i+batch_size]

                        self.collection.add(
                            documents=[chunk["text"] for chunk in batch_chunks],
                            metadatas=[chunk["metadata"] for chunk in batch_chunks],
                            ids=batch_ids
                        )

                    print(f"✅ Added {len(chunks)} chunks from {filename}")
                    total_chunks += len(chunks)
                else:
                    print(f"⚠️ No text extracted from {filename}")

            except Exception as e:
                print(f"❌ Error processing {filename}: {str(e)}")

        print(f"\n🎉 Processing complete! Total chunks added: {total_chunks}")
        self.get_collection_info()

    def query(self, question: str, n_results: int = 5) -> Tuple[str, List[Dict]]:
        """Query the system and generate an answer."""
        if not self._check_initialized():
            return "❌ System not initialized.", []

        try:
            print(f"🔍 Searching for: {question}")

            # Retrieve relevant chunks
            results = self.collection.query(
                query_texts=[question],
                n_results=n_results
            )

            if not results["documents"][0]:
                return "❌ No relevant documents found for your question.", []

            print(f"📄 Found {len(results['documents'][0])} relevant chunks")

            # Prepare context for the model
            context = "\n\n".join([
                f"From {meta['paper_title']} (Page {meta['page_number']}, {meta['section_title']}):\n{text}"
                for text, meta in zip(results["documents"][0], results["metadatas"][0])
            ])

            # Generate answer
            prompt = f"""Based on the following context from research papers, please answer the question.
            If the answer cannot be found in the context, say so clearly.
            Provide a comprehensive answer and cite the sources using the paper titles, page numbers, and section titles provided.

            Context:
            {context}

            Question: {question}

            Answer:"""

            print("🤖 Generating answer with Gemini...")
            response = self.model.generate_content(prompt)

            return response.text, results["metadatas"][0]

        except Exception as e:
            return f"❌ Error generating answer: {str(e)}", []

    def get_collection_info(self):
        """Get information about the current collection."""
        if not self._check_initialized():
            return 0

        try:
            count = self.collection.count()
            print(f"📊 Collection contains {count} document chunks")
            return count
        except Exception as e:
            print(f"❌ Could not retrieve collection information: {e}")
            return 0

    def list_documents(self):
        """List all documents in the collection."""
        if not self._check_initialized():
            return

        try:
            # Get a sample of documents to see what's available
            count = self.collection.count()
            if count == 0:
                print("📚 No documents found in collection")
                return

            results = self.collection.query(
                query_texts=["content"],
                n_results=min(100, count)
            )

            # Extract unique paper titles
            papers = set()
            if results["metadatas"] and results["metadatas"][0]:
                for meta in results["metadatas"][0]:
                    papers.add(meta["paper_title"])

            if papers:
                print("📚 Documents in collection:")
                for i, paper in enumerate(sorted(papers), 1):
                    print(f"{i}. {paper}")
            else:
                print("📚 No documents found in collection")

        except Exception as e:
            print(f"❌ Error listing documents: {str(e)}")

print("✅ Alternative RAG System class defined successfully!")

✅ Alternative RAG System class defined successfully!


In [6]:
# Fix the missing transformers dependency
print("🔧 Installing missing dependencies...")

# Install the required transformers library
!pip install transformers==4.35.0 torch==2.0.1 tokenizers==0.14.1

# Reinstall sentence-transformers with proper dependencies
!pip install --upgrade sentence-transformers==2.5.1

print("✅ Dependencies fixed!")
print("🔄 Please restart your session after this installation.")
print("📝 Go to Runtime → Restart session, then re-run your RAG system code.")

🔧 Installing missing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.0 MB/s eta 0:00:00
   ━━━━━━

✅ Dependencies fixed!
🔄 Please restart your session after this installation.
📝 Go to Runtime → Restart session, then re-run your RAG system code.


In [2]:
# Initialize the RAG system
rag = RAGSystem()

# Process all PDFs in the Dataset folder
print("\n" + "="*50)
print("PROCESSING DOCUMENTS")
print("="*50)
rag.add_documents("/content/Dataset")

# List all processed documents
print("\n" + "="*50)
print("DOCUMENT SUMMARY")
print("="*50)
rag.list_documents()

Initializing RAG System...
✅ Google API Key retrieved from secrets
✅ Gemini model initialized
✅ ChromaDB client initialized
🔄 Trying sentence-transformers embedding...
⚠️ Sentence-transformers failed: module 'numpy' has no attribute 'dtypes'
🔄 Trying Google embedding...
⚠️ Google embedding failed: module 'chromadb.utils.embedding_functions' has no attribute 'GoogleGenerativeAiEmbeddingFunction'
🔄 Trying default embedding...
✅ Default embedding function working
✅ Created new ChromaDB collection
🚀 RAG System ready!

PROCESSING DOCUMENTS
📚 Found 3 PDF files to process

📖 Processing: 2005.14165v4.pdf


Processing 2005.14165v4.pdf: 100%|██████████| 75/75 [00:04<00:00, 17.70it/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:09<00:00, 8.96MiB/s]


✅ Added 332 chunks from 2005.14165v4.pdf

📖 Processing: 2005.11401v4.pdf


Processing 2005.11401v4.pdf: 100%|██████████| 19/19 [00:00<00:00, 105.16it/s]


✅ Added 96 chunks from 2005.11401v4.pdf

📖 Processing: 1706.03762v7.pdf


Processing 1706.03762v7.pdf: 100%|██████████| 15/15 [00:00<00:00, 30.21it/s]


✅ Added 59 chunks from 1706.03762v7.pdf

🎉 Processing complete! Total chunks added: 487
📊 Collection contains 487 document chunks

DOCUMENT SUMMARY
📚 Documents in collection:
1. 1706.03762v7.pdf
2. 2005.11401v4.pdf
3. 2005.14165v4.pdf


In [3]:
def ask_question(question: str, n_results: int = 5):
    """Function to ask questions to the RAG system with nice formatting"""
    print("\n" + "="*80)
    print(f"❓ QUESTION: {question}")
    print("="*80)

    answer, sources = rag.query(question, n_results)

    print("\n💡 ANSWER:")
    print("-" * 40)
    print(answer)

    if sources:
        print(f"\n📚 SOURCES ({len(sources)} chunks):")
        print("-" * 40)
        for i, source in enumerate(sources, 1):
            print(f"{i}. 📄 {source['paper_title']}")
            print(f"   📍 Page {source['page_number']}, Section: {source['section_title']}")
            print()

    print("="*80)

def ask_multiple_questions(questions: List[str]):
    """Process multiple questions at once"""
    for i, question in enumerate(questions, 1):
        print(f"\n🔢 Question {i}/{len(questions)}")
        ask_question(question)

# Example usage function
def run_example_queries():
    """Run some example queries on the uploaded documents"""
    example_questions = [
        "What are the main objectives or goals mentioned in the research?",
        "What methodology or approach was used in the studies?",
        "What are the key findings or results?",
        "What conclusions were drawn from the research?",
        "What limitations or challenges are mentioned?",
        "What future work or recommendations are suggested?"
    ]

    print("🚀 Running example queries...")
    ask_multiple_questions(example_questions)

print("✅ Helper functions ready!")
print("\n🎯 You can now use:")
print("• ask_question('Your question here')")
print("• run_example_queries() - to run predefined questions")
print("• rag.query('question') - direct query method")

✅ Helper functions ready!

🎯 You can now use:
• ask_question('Your question here')
• run_example_queries() - to run predefined questions
• rag.query('question') - direct query method


In [5]:
import os
from datetime import datetime

def interactive_mode():
    """Interactive mode for asking questions"""
    print("\n🤖 RAG SYSTEM INTERACTIVE MODE")
    print("="*50)
    print("Type your questions and get answers from your documents!")
    print("Commands:")
    print("• 'quit' or 'exit' - Exit interactive mode and save responses")
    print("• 'list' - Show all documents")
    print("• 'info' - Show collection info")
    print("• 'help' - Show this help")
    print("="*50)

    # Store all Q&A pairs for saving later
    qa_history = []

    while True:
        try:
            question = input("\n❓ Enter your question: ").strip()

            if question.lower() in ['quit', 'exit', 'q']:
                # Save responses before exiting
                save_responses(qa_history)
                print("👋 Goodbye!")
                break
            elif question.lower() == 'list':
                rag.list_documents()
            elif question.lower() == 'info':
                rag.get_collection_info()
            elif question.lower() == 'help':
                print("Commands: quit/exit, list, info, help")
                print("Or just type your question!")
            elif question:
                # Ask question and store the response
                answer, metadata = ask_question(question)
                if answer and answer != "❌ System not initialized.":
                    qa_history.append({
                        'question': question,
                        'answer': answer,
                        'metadata': metadata,
                        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    })
            else:
                print("Please enter a question or command.")

        except KeyboardInterrupt:
            # Save responses before exiting on Ctrl+C
            save_responses(qa_history)
            print("\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {str(e)}")

def save_responses(qa_history):
    """Save all Q&A responses to a text file in the responses folder"""
    if not qa_history:
        print("📝 No responses to save.")
        return

    try:
        # Create responses folder if it doesn't exist
        response_folder = "/content/responses"
        os.makedirs(response_folder, exist_ok=True)
        print(f"📁 Created/verified responses folder: {response_folder}")

        # Generate filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"rag_responses_{timestamp}.txt"
        filepath = os.path.join(response_folder, filename)

        # Write responses to file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write("RAG SYSTEM - QUESTION & ANSWER SESSION\n")
            f.write("="*60 + "\n\n")
            f.write(f"Session Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Total Questions Asked: {len(qa_history)}\n")
            f.write("="*60 + "\n\n")

            for i, qa in enumerate(qa_history, 1):
                f.write(f"QUESTION {i}:\n")
                f.write(f"Time: {qa['timestamp']}\n")
                f.write(f"Q: {qa['question']}\n")
                f.write("-" * 40 + "\n")
                f.write(f"ANSWER:\n{qa['answer']}\n")

                # Add metadata if available
                if qa['metadata']:
                    f.write("\nSOURCES:\n")
                    for j, meta in enumerate(qa['metadata'], 1):
                        f.write(f"{j}. {meta.get('paper_title', 'Unknown')} "
                               f"(Page {meta.get('page_number', 'N/A')}, "
                               f"{meta.get('section_title', 'N/A')})\n")

                f.write("\n" + "="*60 + "\n\n")

        print(f"✅ Responses saved successfully!")
        print(f"📄 File: {filename}")
        print(f"📂 Location: {response_folder}")
        print(f"💾 Total responses saved: {len(qa_history)}")

        # Also try to trigger download in Colab
        try:
            from google.colab import files
            print(f"📥 Downloading file...")
            files.download(filepath)
        except:
            print("📝 File saved locally. You can find it in the responses folder.")

    except Exception as e:
        print(f"❌ Error saving responses: {str(e)}")

def ask_question(question):
    """Helper function to ask a question and return the response"""
    try:
        answer, metadata = rag.query(question)
        print(f"\n🤖 Answer:\n{answer}")
        return answer, metadata
    except Exception as e:
        error_msg = f"❌ Error getting answer: {str(e)}"
        print(error_msg)
        return error_msg, []

# Start interactive mode
print("🎮 Ready for questions! Starting interactive mode...")
interactive_mode()

🎮 Ready for questions! Starting interactive mode...

🤖 RAG SYSTEM INTERACTIVE MODE
Type your questions and get answers from your documents!
Commands:
• 'quit' or 'exit' - Exit interactive mode and save responses
• 'list' - Show all documents
• 'info' - Show collection info
• 'help' - Show this help

❓ Enter your question: What are the main objectives or goals mentioned in the research?
🔍 Searching for: What are the main objectives or goals mentioned in the research?
📄 Found 5 relevant chunks
🤖 Generating answer with Gemini...

🤖 Answer:
The provided text mentions several research objectives and goals:

* **Improving description of complex scenes:**  One goal, from paper [59] ("Diverse beam search for improved description of complex scenes"), focuses on improving the ability of AI to generate diverse and accurate descriptions of complex scenes.  (2005.11401v4.pdf, Page 15, Content)

* **Creating a multi-task benchmark for natural language understanding:**  Another goal, from paper [60] 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

👋 Goodbye!
